In [1]:
import tensorflow.compat.v1 as tf
from sklearn.metrics import confusion_matrix
import numpy as np
from scipy.io import loadmat
import os
from pywt import wavedec
from functools import reduce
from scipy import signal
from scipy.stats import entropy
from scipy.fft import fft, ifft
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow import keras as K
import matplotlib.pyplot as plt
import scipy
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold,cross_validate
from tensorflow.keras.layers import Dense, Activation, Flatten, concatenate, Input, Dropout, LSTM, Bidirectional,BatchNormalization,PReLU,ReLU,Reshape
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential, Model, load_model
import matplotlib.pyplot as plt;
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.decomposition import PCA
from tensorflow import keras
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Conv1D,Conv2D,Add
from tensorflow.keras.layers import MaxPool1D, MaxPooling2D
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks,layers
import random
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.layers import Dense, Flatten, LSTM, Bidirectional, Input, Dropout, InputLayer, BatchNormalization
from tensorflow.keras.utils import plot_model

In [2]:
data = pd.read_csv("Dataset_Combined.csv")

In [3]:
data

,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,totPwr,status,BVP,ACC_x,ACC_y,ACC_z,EDA,TEMP
0,33448,7054,1692,8833,9796,17692,8025,6926,93466,0,52.20,6,2,65,1.676214,35.55
1,33448,7054,1692,8833,9796,17692,8025,6926,93466,0,49.83,6,2,65,1.676214,35.55
2,33448,7054,1692,8833,9796,17692,8025,6926,93466,0,44.65,6,2,65,1.676214,35.55
3,33448,7054,1692,8833,9796,17692,8025,6926,93466,0,35.26,6,2,65,1.676214,35.55
4,33448,7054,1692,8833,9796,17692,8025,6926,93466,0,22.66,6,2,65,1.676214,35.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287995,1769201,676309,144288,170323,81213,128252,142854,30079,3142519,1,5.81,-51,-13,-37,0.171678,34.09
287996,1769201,676309,144288,170323,81213,128252,142854,30079,3142519,1,6.03,-51,-13,-37,0.171678,34.09
287997,1769201,676309,144288,170323,81213,128252,142854,30079,3142519,1,6.17,-51,-13,-37,0.171678,34.09
287998,1769201,676309,144288,170323,81213,128252,142854,30079,3142519,1,6.30,-51,-13,-37,0.171678,34.09


In [4]:
to_drop = ['ACC_x', 'ACC_y', 'ACC_z']
data.drop(to_drop, axis=1, inplace=True)

In [5]:
y = data.pop('status')
X = data

In [6]:
X = StandardScaler().fit_transform(X)

In [7]:
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
X_train = np.array(X_train).reshape(-1,12,1)
x_test = np.array(x_test).reshape(-1,12,1)

In [8]:
def LSTM_Model(optimizer='adam'):
    model = Sequential()
    model.add(InputLayer(input_shape=(12, 1)))
    model.add(Dense(64, activation='relu'))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    #model.add(Dropout(0.2))
    #model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
batch_size=64
lstm_model = LSTM_Model()
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
lstm_hist = lstm_model.fit(X_train, y_train,
                           validation_data=(x_test, y_test),
                           epochs=25,
                           steps_per_epoch=X_train.shape[0]//batch_size,
                          callbacks=[es])

Epoch 1/25
3825/3825 [==============================] - 78s 19ms/step - loss: 0.5704 - accuracy: 0.6755 - val_loss: 0.4906 - val_accuracy: 0.7401
Epoch 2/25
3825/3825 [==============================] - 71s 19ms/step - loss: 0.4197 - accuracy: 0.7901 - val_loss: 0.3291 - val_accuracy: 0.8424
Epoch 3/25
3825/3825 [==============================] - 70s 18ms/step - loss: 0.2509 - accuracy: 0.8885 - val_loss: 0.1402 - val_accuracy: 0.9449
Epoch 4/25
2533/3825 [==================>...........] - ETA: 21s - loss: 0.1443 - accuracy: 0.9410